In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 패키지 설치
!pip install -q transformers autoawq pillow torch torchvision torchaudio vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

from vllm import LLM, SamplingParams

tokenizer = AutoTokenizer.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq", trust_remote_code=True)
processor = LlavaNextProcessor.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq", do_pad=False)

PROMPT= """You are a versatile AI assistant named Bllava, capable of both understanding and generating text as well as interpreting and analyzing images. Your role is to kindly and effectively answer the user’s questions, whether they are about text or images, and provide appropriate and helpful responses to all types of queries.

당신은 텍스트를 이해하고 생성하는 것뿐만 아니라 이미지를 해석하고 분석할 수 있는 다재다능한 AI 어시스턴트 블라바입니다. 사용자의 질문이 텍스트에 관한 것이든 이미지에 관한 것이든 친절하고 효과적으로 답변하며, 모든 유형의 질의에 대해 적절하고 유용한 응답을 제공하는 것이 당신의 역할입니다."""

instruction = '자연어처리 15주 분량 커리큘럼을 짜줘'

messages = [
    {'role': 'system',
     'content': [
         {"type":"text",
          "text":f"{PROMPT}"}
         ]
     },

    {'role': 'user',
     'content': [
         {"type":"text", "text" : f"{instruction}"}
         ]
     }
]

chat_messages = processor.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt',
)

bos_token = processor.tokenizer.bos_token_id
chat_messages = torch.cat([torch.tensor([[bos_token]]),chat_messages],dim=-1)


# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


In [11]:
# Create an LLM.
llm = LLM(model="ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq",
          tokenizer="ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq",
          trust_remote_code = True,
          dtype="float16",
          quantization="AWQ")

llm.eval()

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
with torch.inference_mode():
  output = llm.generate(input_ids = chat_messages,
                        use_cache=True,
                        max_new_tokens=2048,
                        do_sample=True,
                        sampling_params=sampling_params)

  # Print the outputs.
  print(processor.tokenizer.decode(output[0]))

WARNING 09-23 03:45:40 config.py:1425] Casting torch.bfloat16 to torch.float16.
WARNING 09-23 03:45:40 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 09-23 03:45:40 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq', speculative_config=None, tokenizer='ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-23 03:45:58 model_runner.py:692] Loading model weights took 5.9236 GB
ERROR 09-23 03:45:58 image.py:126] Failed to process image (<PIL.Image.Image image mode=RGB size=448x448 at 0x7F718053ED10>)


UnboundLocalError: local variable 'processed_images' referenced before assignment

llava issue 있음

> UnboundLocalError: local variable 'processed_images' referenced before assignment

https://github.com/haotian-liu/LLaVA/issues/840

## Bllossom LLM (only text)

In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

from vllm import LLM, SamplingParams

tokenizer = AutoTokenizer.from_pretrained("ywhwang/llama-3-Korean-Bllossom-8B-awq", trust_remote_code=True)
PROMPT= """You are a versatile AI assistant named Bllava, capable of both understanding and generating text as well as interpreting and analyzing images. Your role is to kindly and effectively answer the user’s questions, whether they are about text or images, and provide appropriate and helpful responses to all types of queries.

당신은 텍스트를 이해하고 생성하는 것뿐만 아니라 이미지를 해석하고 분석할 수 있는 다재다능한 AI 어시스턴트 블라바입니다. 사용자의 질문이 텍스트에 관한 것이든 이미지에 관한 것이든 친절하고 효과적으로 답변하며, 모든 유형의 질의에 대해 적절하고 유용한 응답을 제공하는 것이 당신의 역할입니다."""

instruction = '자연어처리 15주 분량 커리큘럼을 짜줘'

messages = [
    {'role': 'system', 'content': f"{PROMPT}"},
    {'role': 'user', 'content': f"{instruction}"}
]

chat_messages = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt',
)

bos_token = tokenizer.bos_token_id
chat_messages = torch.cat([torch.tensor([[bos_token]]),chat_messages],dim=-1)


# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [5]:
# 그냥 LLM
# Create an LLM.
# processed_images = ""
llm = LLM(model="ywhwang/llama-3-Korean-Bllossom-8B-awq",
          tokenizer="ywhwang/llama-3-Korean-Bllossom-8B-awq",
          trust_remote_code = True,
          dtype="float16",
          quantization="AWQ")


# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
with torch.inference_mode():
  output = llm.generate(input_ids = chat_messages,
                        use_cache=True,
                        max_new_tokens=2048,
                        do_sample=True,
                        sampling_params=sampling_params)

  # Print the outputs.
  print(tokenizer.decode(output[0]))

WARNING 09-23 03:30:21 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 09-23 03:30:21 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='ywhwang/llama-3-Korean-Bllossom-8B-awq', speculative_config=None, tokenizer='ywhwang/llama-3-Korean-Bllossom-8B-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=ywhwang/llama-3-Korean-Bllossom-8B-awq, use_v2_block_man

OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB. GPU 